## TrainingRunnner

- setup dataloader
- start training

## Import and Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export 
from exp.nb_CustomDataLoader import Detection_dataset

In [ ]:
# !pip3 install tensorboardX

In [ ]:
import os, sys
sys.path.append("/Users/ckh/Documents/Poladrone/nb/ref/pytorchYOLOv4")

In [ ]:
# import torch
# import torch.nn as nn
# from torch.utils.data import DataLoader
# from torch import optim
# from tensorboardX import SummaryWriter
# import logging
# import os, sys
# from tqdm import tqdm
# # from dataset import Yolo_dataset
# # from cfg import Cfg__init
# from ref.pytorchYOLOv4.models import Yolov4

# import argparse
# from easydict import EasyDict as edict
# from torch.nn import functional as F

# import numpy as np

In [ ]:
from ref.pytorchYOLOv4.train import *

## Function

In [ ]:
def custom_collate(batch):
    images = []
    bboxes = []
    for i, (img, box) in enumerate(batch):
        images.append([img])
        
        box = np.concatenate((box, [[i]]*len(box)), 1)
        bboxes.append(box)
        
    images = np.concatenate(images, axis=0)
    images = images.transpose(0, 3, 1, 2)
    images = torch.from_numpy(images).div(255.0)
            
    bboxes = np.concatenate(bboxes)

    bboxes = torch.from_numpy(bboxes)
    return images, bboxes

In [ ]:
def custom_train(train_dataset, val_dataset, model, device, config, epochs=5, batch_size=1, save_cp=True, log_step=20, img_scale=0.5):
#     train_dataset = Yolo_dataset(config.train_label, config)
#     val_dataset = Yolo_dataset(config.val_label, config)

    n_train = len(train_dataset)
    n_val = len(val_dataset)

    train_loader = DataLoader(train_dataset, batch_size=config.batch // config.subdivisions, shuffle=True,
                              num_workers=0, pin_memory=True, drop_last=True, collate_fn=custom_collate)

#     val_loader = DataLoader(val_dataset, batch_size=config.batch // config.subdivisions, shuffle=True, num_workers=1,
#                             pin_memory=True, drop_last=True)

    writer = SummaryWriter(log_dir=config.TRAIN_TENSORBOARD_DIR,
                           filename_suffix=f'OPT_{config.TRAIN_OPTIMIZER}_LR_{config.learning_rate}_BS_{config.batch}_Sub_{config.subdivisions}_Size_{config.width}',
                           comment=f'OPT_{config.TRAIN_OPTIMIZER}_LR_{config.learning_rate}_BS_{config.batch}_Sub_{config.subdivisions}_Size_{config.width}')
    # writer.add_images('legend',
    #                   torch.from_numpy(train_dataset.label2colorlegend2(cfg.DATA_CLASSES).transpose([2, 0, 1])).to(
    #                       device).unsqueeze(0))
    max_itr = config.TRAIN_EPOCHS * n_train
    # global_step = cfg.TRAIN_MINEPOCH * n_train
    global_step = 0
    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {config.batch}
        Subdivisions:    {config.subdivisions}
        Learning rate:   {config.learning_rate}
        Training size:   {n_train}
        Validation size: {n_val}
        Checkpoints:     {save_cp}
        Device:          {device.type}
        Images size:     {config.width}
        Optimizer:       {config.TRAIN_OPTIMIZER}
        Dataset classes: {config.classes}
        Train label path:{config.train_label}
        Pretrained:
    ''')

    # learning rate setup
    def burnin_schedule(i):
        if i < config.burn_in:
            factor = pow(i / config.burn_in, 4)
        elif i < config.steps[0]:
            factor = 1.0
        elif i < config.steps[1]:
            factor = 0.1
        else:
            factor = 0.01
        return factor

    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate / config.batch, betas=(0.9, 0.999), eps=1e-08)
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, burnin_schedule)

    criterion = Yolo_loss(n_classes=config.classes, device=device, batch=config.batch // config.subdivisions)
    # scheduler = ReduceLROnPlateau(optimizer, mode='max', verbose=True, patience=6, min_lr=1e-7)
    # scheduler = CosineAnnealingWarmRestarts(optimizer, 0.001, 1e-6, 20)

    model.train()
    for epoch in range(epochs):
        #model.train()
        epoch_loss = 0
        epoch_step = 0

        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img', ncols=50) as pbar:
            for i, batch in enumerate(train_loader):
                global_step += 1
                epoch_step += 1
                images = batch[0]
                bboxes = batch[1]

                images = images.to(device=device, dtype=torch.float32)
                bboxes = bboxes.to(device=device)

                bboxes_pred = model(images)
                loss, loss_xy, loss_wh, loss_obj, loss_cls, loss_l2 = criterion(bboxes_pred, bboxes)
                # loss = loss / config.subdivisions
                loss.backward()

                epoch_loss += loss.item()

                if global_step  % config.subdivisions == 0:
                    optimizer.step()
                    scheduler.step()
                    model.zero_grad()

                if global_step % (log_step * config.subdivisions) == 0:
                    writer.add_scalar('train/Loss', loss.item(), global_step)
                    writer.add_scalar('train/loss_xy', loss_xy.item(), global_step)
                    writer.add_scalar('train/loss_wh', loss_wh.item(), global_step)
                    writer.add_scalar('train/loss_obj', loss_obj.item(), global_step)
                    writer.add_scalar('train/loss_cls', loss_cls.item(), global_step)
                    writer.add_scalar('train/loss_l2', loss_l2.item(), global_step)
                    writer.add_scalar('lr', scheduler.get_lr()[0] * config.batch, global_step)
                    pbar.set_postfix(**{'loss (batch)': loss.item(), 'loss_xy': loss_xy.item(),
                                        'loss_wh': loss_wh.item(),
                                        'loss_obj': loss_obj.item(),
                                        'loss_cls': loss_cls.item(),
                                        'loss_l2': loss_l2.item(),
                                        'lr': scheduler.get_lr()[0] * config.batch
                                        })
                    logging.debug('Train step_{}: loss : {},loss xy : {},loss wh : {},'
                                  'loss obj : {}，loss cls : {},loss l2 : {},lr : {}'
                                  .format(global_step, loss.item(), loss_xy.item(),
                                          loss_wh.item(), loss_obj.item(),
                                          loss_cls.item(), loss_l2.item(),
                                          scheduler.get_lr()[0] * config.batch))

                pbar.update(images.shape[0])

            if save_cp:
                try:
                    os.mkdir(config.checkpoints)
                    logging.info('Created checkpoint directory')
                except OSError:
                    pass
                torch.save(model.state_dict(), os.path.join(config.checkpoints, f'Yolov4_epoch{epoch + 1}.pth'))
                logging.info(f'Checkpoint {epoch + 1} saved !')

    writer.close()

## Main

### Build DataLoader

In [ ]:
from config.config import Cfg
data_list = "/Users/ckh/OneDrive - Default Directory/Hui_Wan/train_npt.txt"
dataset = Detection_dataset(data_list, Cfg)
dataset.names


## without augmentation
val_dataset = Detection_dataset(data_list, Cfg)

In [ ]:
logging = init_logger(log_dir='log', stdout=False)

In [ ]:
cfg = get_args(**Cfg)
os.environ["CUDA_VISIBLE_DEVICES"] = cfg.gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')

model = Yolov4(cfg.pretrained,n_classes=cfg.classes)

if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)
model.to(device=device)

print()

### Train

In [ ]:
import random
import torch
torch.manual_seed(20)
np.random.seed(20)
random.seed(20)


try:
    custom_train(
        train_dataset=dataset,
        val_dataset=val_dataset,
        model=model,
        config=cfg,
        epochs=cfg.TRAIN_EPOCHS,
        device=device, )
except KeyboardInterrupt:
    torch.save(model.state_dict(), 'INTERRUPTED.pth')
    logging.info('Saved interrupt')
    try:
        sys.exit(0)
    except SystemExit:
        os._exit(0)

In [ ]:
import random
import torch
torch.manual_seed(20)
np.random.seed(20)
random.seed(20)

[len(dataset[i][1]) for i in range(5)]

## Export

In [ ]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "NOTEBOOK = " + "'"+thename+"'";
kernel.execute(command);

In [ ]:
!python notebook2script.py "$NOTEBOOK".ipynb

In [ ]:
%%javascripddt
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "NOTEBOOK = " + "'"+thename+"'";
kernel.execute(command);

!python notebook2script.py "$NOTEBOOK".ipynb